**Course**: [BIO-341 Systèmes Dynamiques en Biologie](https://moodle.epfl.ch/course/info.php?id=14291)

**Professor**: _Felix Naef_
# Phase portraits for linear sytems in 2D

This notebook illustrate phase portraits for the linear system $$\dot{x}=A x$$ with a matrix $A$ of the type
$$A=\begin{bmatrix}
-1 & 3 \\
-\epsilon & -1
\end{bmatrix}$$
In the notebook below, you can change $\epsilon$. For $\epsilon=0$, the matrix is non-diagonalizable and the eigenspace for the eigenvalue $\lambda=-1$ is one-dimensional. This leads to a so-called degenerate fixed point. For $\epsilon\ne0$ there are two distinct eigenvalues.

Here we show the solution using the exact expression
\begin{equation}
e^{tA}=e^{st}\left(\left(\cosh(qt)-s\frac{\sinh(qt)}{q}\right)I + \frac{\sinh(qt)}{q} A\right)
\end{equation}
with $s=\frac 1 2 (\alpha+\beta)$ and $q=\frac 1 2 (\alpha-\beta)$ and $\alpha,\beta$ the eigenvalues of $A$. 

In [5]:
#import important libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from scipy.integrate import odeint
from IPython.display import set_matplotlib_formats
from matplotlib.markers import MarkerStyle
set_matplotlib_formats('png', 'pdf')

## Phase portrait

Plots the phase portrait in function of $\epsilon$. Eigenvalues are shown on top of the plot.

In [6]:
def exp_tA(A, t):
    [Lambda, V] = np.linalg.eig(A)
    s=(Lambda[0]+Lambda[1])/2
    q=(Lambda[0]-Lambda[1])/2
#     print((s,q))
    
    f1=np.exp(s*t)
    if np.abs(q)>0:
        f2=np.sinh(q*t)/q
    else:
        f2=t
    f3=np.cosh(q*t)-s*f2
#     print((f1,f2,f3))
    
    exptA = f1 * (f3*np.eye(2) + f2 * A)
    return exptA

def exact_solution(A, x0, times=np.array([1])):
    sol = np.empty((0,2)) # create an empty array
    for t in times:
        xt = exp_tA(A, t) @ np.asarray(x0)
        sol = np.vstack((sol, xt))
    return sol

def phase_portrait(A, xdomain, ydomain, initial_conditions = None, time_domain = None):
    
    #grid for both coordinates
    x, y = np.meshgrid(xdomain, ydomain)
    
    #compute the derivatives for all points of the grid
    dx = A[0,0]*x+A[0,1]*y 
    dy = A[1,0]*x+A[1,1]*y

    #normalize
    temp = dx/(dx**2+dy**2)**0.5 
    dy = dy/(dx**2+dy**2)**0.5
    dx = temp

    #plot the vectorfield
    plt.figure(figsize = (15,15)) #initialize an empty square figure
    plt.quiver(x,y,dx,dy, color = "lightblue") #plot the vectorfield
    
    #plot the nullclines  
    plt.plot(xdomain, -A[0,0]/A[0,1]*xdomain, c = 'blue', ls = '--', lw = 2, label = 'x-nullcline')
    plt.plot(xdomain, -A[1,0]/A[1,1]*xdomain, c = 'green', ls = '--', lw = 2, label = 'y-nullcline')
    
    #plot the eigenvectors
    [Lambda, V] = np.linalg.eig(A)
    v1, v2 = V[:,0], V[:,1]
    if not abs(Lambda[0].imag)>0:
        plt.plot(xdomain*v1[0], ydomain*v1[1], c = 'orange', ls = '-.', lw = 2, label = 'v1')
        plt.plot(xdomain*v2[0], ydomain*v2[1], c = 'brown', ls = '-.', lw = 2, label = 'v2')

    if initial_conditions is not None and time_domain is not None:
        def F(X, t, A):
            return A@X
        #plot trajectories
        for x0 in initial_conditions:
            X = odeint(F, x0, time_domain, args=(A,))
#             print(X.shape)
            plt.plot(X[:,0],X[:,1], color = 'red')
            
            X_exact = exact_solution(A, x0, times=time_domain)
#             print(X_exact.shape)
            plt.plot(X_exact[:,0],X_exact[:,1], color = 'pink')

    #adjust figure parameters
    plt.xlim([xdomain[0],xdomain[-1]])
    plt.ylim([ydomain[0],ydomain[-1]])
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(str(Lambda[0]) + ' ' + str(Lambda[1]))
    plt.legend()
    plt.grid()
    plt.show()

#define the time domain
time_domain = np.linspace(0,10,1000)

#initial conditions
x0_conditions = [(-10,2),(-10,4),(10,-2),(10,-4)]
x0_conditions = [(-10,2),(-10,4),(10,-2),(10,-4),(-10,5),(-10,6),(-10,8),(-10,10),(10,5),(10,6),(10,8),(10,10)]


def deg_node(x0, time, eps=0.01, range=3):
    #define the function domain
    xdom = np.linspace(-range,range,20)
    ydom = np.linspace(-range,range,20)
    phase_portrait(np.array([[-1,3],[-eps,-1]]), xdom, ydom, x0, time)


interact(deg_node, x0=fixed(x0_conditions), time=fixed(time_domain), eps = (-0.5,0.5,0.01), range=(1,10))


interactive(children=(FloatSlider(value=0.01, description='eps', max=0.5, min=-0.5, step=0.01), IntSlider(valu…

<function __main__.deg_node(x0, time, eps=0.01, range=3)>